In [1]:
import numpy as np
import os

In [2]:
with open('SURFACE','r') as f:
    SURFACE = f.readlines()
surf = np.array([np.fromstring(line, sep='\t', count=3)
                      for line in SURFACE])
with open('H1','r') as f:
    mol = f.readlines()
H1 = np.array([np.fromstring(line, sep='\t', count=3)
                      for line in mol])
with open('H2','r') as f:
    mol = f.readlines()
H2 = np.array([np.fromstring(line, sep='\t', count=3)
                      for line in mol])
with open('H3','r') as f:
    mol = f.readlines()
H3 = np.array([np.fromstring(line, sep='\t', count=3)
                      for line in mol])

In [3]:
Valley = surf[39][:2]
Facet = surf[45][:2]
Ridge = surf[50][:2]

In [4]:
z_Valley = surf[39][2]
z_Facet = surf[45][2]
z_Ridge = surf[50][2]

In [5]:
print(f'Facet : {z_Facet}', f'z_Ridge: {z_Ridge} ', f'z_Valley: {z_Valley}')

Facet : 9.99363 z_Ridge: 11.13482  z_Valley: 8.75599


In [6]:
def calcular_CM_xy(CH4):
    m_H = 1
    m_C = 12.011
    M = 4 * m_H + m_C
    CM = [0,0,0]
    CM[0] = (CH4[0][0] + CH4[1][0] + CH4[2][0] + CH4[3][0] + m_C * CH4[4][0]) / M
    CM[1] = (CH4[0][1] + CH4[1][1] + CH4[2][1] + CH4[3][1] + m_C * CH4[4][1]) / M
    return CM

In [7]:
print(Valley)
print(Facet)
print(Ridge)
print(H1[4])

[2.00000e-05 5.68517e+00]
[6.05984 4.26385]
[4.01998 5.68513]
[ 1.98025707  4.26328258 20.33408   ]


In [8]:
calcular_CM_xy(H1)

[1.980294146723971, 4.263277915835869, 0]

In [9]:
H1_VALLEY = H1[:] - calcular_CM_xy(H1) + [*Valley,0]
H1_VALLEY

array([[ 1.61934760e-05,  4.65509377e+00,  2.07036260e+01],
       [ 4.73364760e-05,  5.68008265e+00,  1.92398076e+01],
       [-8.93385914e-01,  6.20276942e+00,  2.06964749e+01],
       [ 8.93847683e-01,  6.20267818e+00,  2.06963263e+01],
       [-1.70742240e-05,  5.68517466e+00,  2.03340800e+01]])

In [10]:
capa = ['Valley', 'Facet', 'Ridge']

CH4_1 = {}
CH4_2 = {}
CH4_3 = {}
for name, atom in zip(capa,[Valley, Facet, Ridge]):
    CH4_1[name] = H1[:] - calcular_CM_xy(H1) + [*atom,0]
    CH4_2[name] = H2[:] - calcular_CM_xy(H2) + [*atom,0]
    CH4_3[name] = H3[:] - calcular_CM_xy(H3) + [*atom,0]

In [11]:
CH4_1['Valley']

array([[ 1.61934760e-05,  4.65509377e+00,  2.07036260e+01],
       [ 4.73364760e-05,  5.68008265e+00,  1.92398076e+01],
       [-8.93385914e-01,  6.20276942e+00,  2.06964749e+01],
       [ 8.93847683e-01,  6.20267818e+00,  2.06963263e+01],
       [-1.70742240e-05,  5.68517466e+00,  2.03340800e+01]])

In [12]:
header = '''1.0
 +8.0400000000  +0.0000000000  +0.0000000000 
 +0.0000000000  +8.5277077811  +0.0000000000 
 +0.0000000000  +0.0000000000 +31.3702770415 
 51 4 1
Selective
Cartesian'''

In [13]:

# Define file versions
orientacion = ['H1', 'H2', 'H3']

# Create folders for each combination
for atom in capa:
    for direction, CH4 in zip(orientacion, [CH4_1, CH4_2, CH4_3]):
        folder_name = f'source/{direction}_{atom}'
        os.makedirs(folder_name, exist_ok=True)

        output_file_path = os.path.join(folder_name, f'POSCAR')

        # Convert numpy arrays to strings with tabs between columns
        surf_str = '\n'.join('\t'.join('{:.10f}'.format(value) for value in row) for row in surf)
        ch4_str = '\n'.join('\t'.join('{:.10f}'.format(value) for value in row) for row in CH4[atom].tolist())

        # Combine content and write to the output file
        with open(output_file_path, 'w') as output_file:
            output_file.write(f'{header}\n{surf_str}\n{ch4_str}')

        print(f'File created: {output_file_path}')


File created: source/H1_Valley\POSCAR
File created: source/H2_Valley\POSCAR
File created: source/H3_Valley\POSCAR
File created: source/H1_Facet\POSCAR
File created: source/H2_Facet\POSCAR
File created: source/H3_Facet\POSCAR
File created: source/H1_Ridge\POSCAR
File created: source/H2_Ridge\POSCAR
File created: source/H3_Ridge\POSCAR
